# Lesson 1: Execution Trace

When any code executes in the RISC Zero virtual machine, each step of that execution is recorded in an Execution Trace.

In this lesson, we'll look at a simplified example: computing 4 steps of a Fibonacci sequence modulo 97, using two user-specified input values.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def create_fibonacci_trace(a0, a1, steps=4, modulus=97):
    """Creates an execution trace for the Fibonacci sequence.
    
    Args:
        a0 (int): First initial value
        a1 (int): Second initial value
        steps (int): Number of steps
        modulus (int): Modulus for calculations
        
    Returns:
        pd.DataFrame: Execution trace
    """
    # Initialize data
    data = {
        'Register 1': [a0] + [0] * (steps-1),
        'Register 2': [a1] + [0] * (steps-1),
        'Register 3': [0] * steps,
        'Initialize': [1] + [0] * (steps-1),
        'Execute': [0] + [1] * (steps-2) + [0],
        'Terminate': [0] * (steps-1) + [1]
    }
    
    # Calculate Fibonacci sequence
    for i in range(1, steps):
        data['Register 1'][i] = data['Register 2'][i-1]
        data['Register 2'][i] = data['Register 3'][i-1]
        data['Register 3'][i] = (data['Register 1'][i-1] + data['Register 2'][i-1]) % modulus
    
    return pd.DataFrame(data)

Let's create an execution trace for a specific example:

In [ ]:
# Create trace with initial values 3 and 5
trace = create_fibonacci_trace(3, 5)
print("Execution Trace:")
display(trace)

# Visualize the trace
plt.figure(figsize=(12, 6))
for column in trace.columns[:3]:  # Only for registers
    plt.plot(trace.index, trace[column], marker='o', label=column)
plt.title('Register Values During Execution')
plt.xlabel('Step')
plt.ylabel('Value')
plt.grid(True)
plt.legend()
plt.show()

## Understanding the Execution Trace

In our example, the trace consists of 6 columns:

1. **Data Columns** (first three columns):
   - Each of the first three columns records the internal state of a register at each clock cycle from initialization until termination
   
2. **Control Columns** (last three columns):
   - Used to mark initialization and termination points

### In the full RISC Zero protocol:

- **Data Columns** contain the RISC-V processor state, including:
  - ISA registers
  - Program counter
  - Various microarchitecture details
    - Instruction decoding data
    - ALU registers
    - etc.

- **Control Columns** handle:
  - System initialization
  - Shutdown
  - Page table setup with program memory image
  - Other control signals that don't depend on program execution